In [20]:
import numpy as np
import sklearn
import pandas as pd
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import GridSearchCV
#import warnings
#warnings.filterwarnings('ignore')

In [21]:
energy_data = pd.read_csv("energy_efficiency/ENB2012_data.csv").fillna(0)
energy_data = energy_data.drop(["Unnamed: 10", "Unnamed: 11"], axis=1)

In [24]:
len(energy_data)

1296

In [26]:
energy_data.iloc[250:255].reset_index(drop=True)

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
0,0.86,588.0,294.0,147.0,7.0,4.0,0.1,5.0,25.36,32.04
1,0.86,588.0,294.0,147.0,7.0,5.0,0.1,5.0,26.45,31.28
2,0.82,612.5,318.5,147.0,7.0,2.0,0.1,5.0,24.37,25.11
3,0.82,612.5,318.5,147.0,7.0,3.0,0.1,5.0,23.89,24.77
4,0.82,612.5,318.5,147.0,7.0,4.0,0.1,5.0,22.89,28.88


In [4]:

data_x6 = energy_data["X6"].astype("str")
data_x8 = energy_data["X8"].astype("str")

energy_data = energy_data.drop(["X6", "X8"], axis=1)

In [5]:
def closs_val_test(models, params, x, y, ss_y = None):
    kf = KFold(n_splits=5, shuffle=True, random_state=2018)
    result = []
    predict_values = []
    for train, test in kf.split(x):
        grid = GridSearchCV(models, params, cv=5)
        search_result = grid.fit(x[train], y.reshape(-1)[train])
        best = search_result.best_estimator_
        output = best.predict(x[test])
        y_true = y[test]
        if ss_y is not None:
            output = ss_y.inverse_transform(output.reshape(-1,1))
            y_true = ss_y.inverse_transform(y_true.reshape(-1,1))
        result.append(mean_squared_error(y_true, output))
        predict_values.append(output*1)
        print ("best_model:", best)
        print ("output_values:", output)
        print ("mse:", mean_squared_error(y_true, output))
    return result, predict_values

In [6]:
models = SVR()
params = {'kernel':['rbf'] ,'C': [10**i for i in range(-3, 3)],  'gamma' : [2**i for i in range(-5,5)]}
x = energy_data.iloc[:,:-2].values
y1 = energy_data.iloc[:,-2].values.reshape(-1,1)
y2 = energy_data.iloc[:,-1].values.reshape(-1,1)

lb = LabelBinarizer()
lb.fit(data_x6.values)
l_x6 = lb.transform(data_x6.values)
lb.fit(data_x8.values)
l_x8 = lb.transform(data_x8.values)

ss_x = StandardScaler().fit(x)
x_trans = ss_x.transform(x)
ss_y1 = StandardScaler().fit(y1)
y1_trans = ss_y1.transform(y1)
ss_y2 = StandardScaler().fit(y2)
y2_trans = ss_y2.transform(y2)

In [7]:
x_trans = np.hstack([x_trans, l_x6, l_x8])

In [8]:
result_svr_y1, values_svr_y1 = closs_val_test(models, params, x_trans, y1_trans, ss_y1)
result_svr_y2, values_svr_y2 = closs_val_test(models, params, x_trans, y2_trans, ss_y2)

best_model: SVR(C=10, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.0625,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
output_values: [[16.77192429]
 [17.5915187 ]
 [18.68543447]
 [19.13306158]
 [18.02133131]
 [21.25658355]
 [21.58883842]
 [ 6.58118493]
 [ 7.60261885]
 [ 9.27282105]
 [ 9.18589924]
 [ 9.13092992]
 [24.96990098]
 [25.47365277]
 [24.80377229]
 [28.39361842]
 [27.58213035]
 [28.17621497]
 [30.01004279]
 [34.27863152]
 [33.26314672]
 [13.01259639]
 [13.80379286]
 [13.72431821]
 [13.68739578]
 [14.40490047]
 [14.25836466]
 [24.65426131]
 [25.05149728]
 [25.58807815]
 [26.59644287]
 [29.53612226]
 [33.4027148 ]
 [13.00436305]
 [13.16914206]
 [14.5125435 ]
 [29.24667529]
 [29.12816305]
 [29.92286638]
 [10.37864316]
 [10.06154795]
 [11.26109074]
 [12.11060986]
 [13.13683804]
 [13.50436657]
 [13.56107079]
 [25.69178987]
 [28.06962276]
 [29.65972356]
 [29.17040765]
 [33.41009098]
 [10.12882947]
 [10.81497506]
 [11.26845679]
 [25.43742047]
 [2

best_model: SVR(C=10, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.03125,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
output_values: [[19.64217662]
 [19.5034557 ]
 [20.49525221]
 [20.3633674 ]
 [25.77339763]
 [25.31179369]
 [ 7.4740143 ]
 [ 8.25002667]
 [ 9.12487608]
 [26.15442886]
 [27.67415813]
 [29.82898844]
 [29.23402828]
 [34.03876089]
 [ 9.73595674]
 [10.88560707]
 [12.13663372]
 [11.90418249]
 [12.03690817]
 [12.92418843]
 [26.11548149]
 [28.18401305]
 [27.78989896]
 [10.54364619]
 [10.87258996]
 [12.57066838]
 [13.57659114]
 [13.42354316]
 [13.56879335]
 [14.25656132]
 [27.15450942]
 [26.75883797]
 [25.8725742 ]
 [27.91911353]
 [28.48081109]
 [33.65302639]
 [32.76521909]
 [ 9.75992423]
 [11.12863117]
 [11.17997495]
 [11.69875732]
 [13.78470851]
 [14.44489384]
 [26.06956335]
 [29.26440528]
 [33.75710516]
 [33.68818602]
 [ 9.51224709]
 [11.89762194]
 [11.87792418]
 [12.91207782]
 [13.48442098]
 [13.67435441]
 [14.24911971]
 [14.21983674]
 [

best_model: SVR(C=10, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.03125,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
output_values: [[13.36349873]
 [14.16760521]
 [14.89537172]
 [13.94442925]
 [14.05429336]
 [15.67286143]
 [14.8958792 ]
 [14.79705982]
 [30.26241381]
 [28.05119244]
 [29.34905757]
 [12.38044791]
 [14.58241088]
 [16.35080624]
 [16.03116562]
 [16.56667127]
 [25.95777881]
 [28.67589784]
 [29.45059932]
 [35.03555449]
 [33.96077608]
 [12.86582093]
 [11.89013734]
 [15.06459163]
 [26.4123274 ]
 [26.39181793]
 [29.14760002]
 [28.25699418]
 [13.57564877]
 [14.3986505 ]
 [15.04312194]
 [14.85282403]
 [15.78572622]
 [15.7434401 ]
 [15.43804857]
 [27.35074234]
 [27.33871151]
 [29.2139806 ]
 [29.41975137]
 [30.73334122]
 [13.5831798 ]
 [14.343534  ]
 [15.07139638]
 [16.50161315]
 [16.19855533]
 [28.16013168]
 [28.72755511]
 [27.82683458]
 [30.67321902]
 [31.95777455]
 [34.66155692]
 [35.40124852]
 [13.99975645]
 [14.68304122]
 [15.31902281]
 [

best_model: SVR(C=10, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.125,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
output_values: [[26.05745497]
 [30.28745538]
 [27.19967892]
 [30.36604424]
 [10.06533525]
 [10.58750297]
 [13.27619699]
 [13.81024401]
 [24.5644701 ]
 [13.00517669]
 [14.2714775 ]
 [14.02596369]
 [15.51116184]
 [16.39652149]
 [24.79609905]
 [30.97404721]
 [29.1691275 ]
 [30.26689098]
 [30.21594459]
 [32.53051933]
 [34.64163317]
 [12.76449517]
 [14.22588201]
 [14.21468548]
 [15.47942945]
 [15.6915247 ]
 [15.61610221]
 [16.83901696]
 [27.64869451]
 [24.98260912]
 [31.77339649]
 [26.48153124]
 [31.46734055]
 [15.0748073 ]
 [28.36202065]
 [30.86014684]
 [34.67692805]
 [33.97011157]
 [34.9572788 ]
 [12.78844107]
 [16.02177018]
 [16.35912861]
 [25.09593531]
 [28.99872563]
 [28.4725358 ]
 [33.96803317]
 [36.23866944]
 [12.62958257]
 [14.78097575]
 [15.56427579]
 [16.15171003]
 [32.36130854]
 [32.93168079]
 [35.58402624]
 [33.28089542]
 [37

In [9]:
print ("svr y1", np.mean(result_svr_y1))
print ("svr y2", np.mean(result_svr_y2))

svr y1 4.754936717989236
svr y2 6.310870863172655


In [10]:
from sklearn.linear_model import Lasso

In [11]:
models_l = Lasso()
params = {'alpha': [10**i for i in range(-3, 3)]}
result_y1, values_y1 = closs_val_test(models_l, params, x_trans, y1_trans, ss_y1)
result_y2, values_y2 = closs_val_test(models_l, params, x_trans, y2_trans, ss_y2)

best_model: Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)
output_values: [[ 2.12788502e+01]
 [ 2.13195052e+01]
 [ 2.29808790e+01]
 [ 2.22238379e+01]
 [ 2.22238379e+01]
 [ 2.35837675e+01]
 [ 2.47639671e+01]
 [ 4.27315265e+00]
 [ 5.27362243e+00]
 [ 7.41363685e+00]
 [ 7.45429188e+00]
 [ 7.45429188e+00]
 [ 2.68956151e+01]
 [ 2.69362701e+01]
 [ 2.69362701e+01]
 [ 2.85976439e+01]
 [ 2.85976439e+01]
 [ 2.91598774e+01]
 [ 3.03400771e+01]
 [ 3.36779024e+01]
 [ 3.36779024e+01]
 [ 1.20705871e+01]
 [ 1.30304018e+01]
 [ 1.30710568e+01]
 [ 1.30710568e+01]
 [ 1.40308716e+01]
 [ 1.40715266e+01]
 [ 2.68936077e+01]
 [ 2.69342627e+01]
 [ 2.77979404e+01]
 [ 2.78385954e+01]
 [ 3.03380696e+01]
 [ 3.36758950e+01]
 [ 1.20685796e+01]
 [ 1.20685796e+01]
 [ 1.40695192e+01]
 [ 2.90262858e+01]
 [ 2.90262858e+01]
 [ 3.01658305e+01]
 [ 8.53547136e+00]
 [ 8.53547136e+00]
 

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increa

best_model: Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)
output_values: [[ 3.96624233e+00]
 [ 4.96947665e+00]
 [ 6.17327836e+00]
 [ 6.17327836e+00]
 [ 6.17327836e+00]
 [ 8.17974699e+00]
 [ 8.17974699e+00]
 [ 8.11634475e+00]
 [ 2.83079257e+01]
 [ 2.83079257e+01]
 [ 2.77806762e+01]
 [ 8.66719155e+00]
 [ 9.87099327e+00]
 [ 1.30812636e+01]
 [ 1.40210957e+01]
 [ 1.40844979e+01]
 [ 2.64015945e+01]
 [ 2.82445234e+01]
 [ 2.77806762e+01]
 [ 3.35242673e+01]
 [ 3.34608651e+01]
 [ 8.66719155e+00]
 [ 8.66719155e+00]
 [ 1.30812636e+01]
 [ 2.64649968e+01]
 [ 2.64015945e+01]
 [ 2.83079257e+01]
 [ 2.77806762e+01]
 [ 9.80759103e+00]
 [ 1.08742276e+01]
 [ 1.20146271e+01]
 [ 1.20780293e+01]
 [ 1.40844979e+01]
 [ 1.40210957e+01]
 [ 1.40844979e+01]
 [ 2.65578079e+01]
 [ 2.65578079e+01]
 [ 2.78734873e+01]
 [ 2.78100850e+01]
 [ 2.92778564e+01]
 [ 8.76000265e+00]
 

best_model: Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)
output_values: [[ 2.23107222e+01]
 [ 2.42808495e+01]
 [ 2.42808495e+01]
 [ 2.77156464e+01]
 [ 2.45251175e+00]
 [ 2.45251175e+00]
 [ 4.76712677e+00]
 [ 8.14208754e+00]
 [ 2.65236590e+01]
 [ 8.47157036e+00]
 [ 9.72583963e+00]
 [ 9.72583963e+00]
 [ 1.07861854e+01]
 [ 1.20404547e+01]
 [ 2.65236590e+01]
 [ 2.83297808e+01]
 [ 2.83297808e+01]
 [ 2.90456388e+01]
 [ 2.90456388e+01]
 [ 3.02999081e+01]
 [ 3.37347050e+01]
 [ 8.47157036e+00]
 [ 9.72583963e+00]
 [ 9.72583963e+00]
 [ 1.07861854e+01]
 [ 1.07861854e+01]
 [ 1.07861854e+01]
 [ 1.41611461e+01]
 [ 2.65236590e+01]
 [ 2.65236590e+01]
 [ 2.83297808e+01]
 [ 2.75974460e+01]
 [ 3.02999081e+01]
 [ 1.07861854e+01]
 [ 2.65236590e+01]
 [ 2.90456388e+01]
 [ 3.02999081e+01]
 [ 3.02999081e+01]
 [ 3.37347050e+01]
 [ 8.47157036e+00]
 [ 1.07861854e+01]
 

best_model: Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)
output_values: [[ 2.78243906e+01]
 [ 2.78243906e+01]
 [ 2.90142289e+01]
 [ 2.90142289e+01]
 [ 3.22172799e+01]
 [ 3.24307318e+01]
 [ 1.15956953e+01]
 [ 1.23276257e+01]
 [ 1.34498232e+01]
 [ 3.02882053e+01]
 [ 3.17527195e+01]
 [ 3.25519108e+01]
 [ 3.27653627e+01]
 [ 3.47423186e+01]
 [ 1.19858934e+01]
 [ 1.29985365e+01]
 [ 1.39072821e+01]
 [ 1.38400213e+01]
 [ 1.41207340e+01]
 [ 1.49199252e+01]
 [ 3.02356698e+01]
 [ 3.14867321e+01]
 [ 3.17001840e+01]
 [ 1.29460010e+01]
 [ 1.32267137e+01]
 [ 1.48001289e+01]
 [ 1.57088745e+01]
 [ 1.56416137e+01]
 [ 1.59223264e+01]
 [ 1.65503592e+01]
 [ 2.99953284e+01]
 [ 3.02760411e+01]
 [ 3.00686260e+01]
 [ 3.12524275e+01]
 [ 3.24785226e+01]
 [ 3.44554785e+01]
 [ 3.46689305e+01]
 [ 1.19797660e+01]
 [ 1.27789572e+01]
 [ 1.27789572e+01]
 [ 1.35531812e+01]
 

In [12]:
print ("Lasso y1", np.mean(result_y1))
print ("Lasso y2", np.mean(result_y2))

Lasso y1 4.862063013332328
Lasso y2 6.227507314765892


In [13]:
from  sklearn.ensemble import RandomForestRegressor
model_f = RandomForestRegressor()
params = {'n_estimators': list(range(5,100,5)), 'min_samples_split' : list(range(2,10))}
result_f_y1, values_f_y1 = closs_val_test(model_f, params, x_trans, y1_trans, ss_y1)
result_f_y2, values_f_y2 = closs_val_test(model_f, params, x_trans, y2_trans, ss_y2)

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


best_model: RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=9,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
output_values: [[ 1.83513435e+01]
 [ 1.83513435e+01]
 [ 1.87286768e+01]
 [ 1.87286768e+01]
 [ 1.87286768e+01]
 [ 1.84709625e+01]
 [ 2.61944667e+01]
 [ 6.34905873e+00]
 [ 6.89672931e+00]
 [ 9.61926205e+00]
 [ 9.61926205e+00]
 [ 9.61926205e+00]
 [ 2.44012106e+01]
 [ 2.42882106e+01]
 [ 2.44012106e+01]
 [ 2.88368937e+01]
 [ 2.86946374e+01]
 [ 2.36964268e+01]
 [ 3.67053500e+01]
 [ 3.26387213e+01]
 [ 3.27629922e+01]
 [ 1.15855481e+01]
 [ 1.52450330e+01]
 [ 1.52558512e+01]
 [ 1.52490377e+01]
 [ 1.29063566e+01]
 [ 1.29191343e+01]
 [ 2.43051533e+01]
 [ 2.43051533e+01]
 [ 2.61270485e+01]
 [ 2.60805152e+01]
 

best_model: RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=3,
           min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
output_values: [[ 1.71507321e+01]
 [ 1.68896667e+01]
 [ 1.89013250e+01]
 [ 1.94468071e+01]
 [ 1.72952833e+01]
 [ 2.88798250e+01]
 [ 2.52032000e+01]
 [ 6.15906667e+00]
 [ 6.14281667e+00]
 [ 6.31312500e+00]
 [ 6.31312500e+00]
 [ 2.62667750e+01]
 [ 2.62435917e+01]
 [ 2.30449583e+01]
 [ 2.31885250e+01]
 [ 3.68555417e+01]
 [ 3.25714250e+01]
 [ 1.04117917e+01]
 [ 1.15854583e+01]
 [ 2.87962917e+01]
 [ 3.65347083e+01]
 [ 3.61773333e+01]
 [ 1.04315708e+01]
 [ 1.14487917e+01]
 [ 1.14333333e+01]
 [ 1.29287500e+01]
 [ 2.56508167e+01]
 [ 2.39555833e+01]
 [ 3.65947667e+01]
 [ 3.31284048e+01]
 [ 3.31426548e+01]
 

best_model: RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=6,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
output_values: [[ 1.94482893e+01]
 [ 2.66532000e+01]
 [ 2.66532000e+01]
 [ 2.64864500e+01]
 [ 6.24662500e+00]
 [ 6.24662500e+00]
 [ 6.81770119e+00]
 [ 9.08694524e+00]
 [ 2.45535429e+01]
 [ 1.03912095e+01]
 [ 1.07255044e+01]
 [ 1.07142960e+01]
 [ 1.11890913e+01]
 [ 1.16315190e+01]
 [ 2.42910502e+01]
 [ 2.89256250e+01]
 [ 2.87364583e+01]
 [ 2.33441353e+01]
 [ 2.31790714e+01]
 [ 3.66433000e+01]
 [ 3.28676667e+01]
 [ 1.04205833e+01]
 [ 1.06958794e+01]
 [ 1.06894627e+01]
 [ 1.12782024e+01]
 [ 1.12782024e+01]
 [ 1.12728214e+01]
 [ 1.29706155e+01]
 [ 2.42910502e+01]
 [ 2.42844788e+01]
 [ 2.92476389e+01]
 

best_model: RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=4,
           min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
output_values: [[ 1.16273667e+01]
 [ 1.18315333e+01]
 [ 1.19300333e+01]
 [ 1.19840333e+01]
 [ 1.19300333e+01]
 [ 1.20400000e+01]
 [ 1.20400000e+01]
 [ 1.20400000e+01]
 [ 3.11453333e+01]
 [ 3.13277333e+01]
 [ 2.71035000e+01]
 [ 1.34921667e+01]
 [ 1.40720000e+01]
 [ 1.93692000e+01]
 [ 1.43960000e+01]
 [ 1.43810000e+01]
 [ 2.59461333e+01]
 [ 3.23545667e+01]
 [ 2.77412500e+01]
 [ 3.39523333e+01]
 [ 3.38366667e+01]
 [ 1.36471000e+01]
 [ 1.35708000e+01]
 [ 1.92346667e+01]
 [ 2.59078333e+01]
 [ 2.60126333e+01]
 [ 3.12917000e+01]
 [ 2.69692500e+01]
 [ 1.39326952e+01]
 [ 1.40980667e+01]
 [ 1.40182667e+01]
 [

best_model: RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=95, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
output_values: [[ 2.71147368e+01]
 [ 2.56383158e+01]
 [ 2.40674737e+01]
 [ 2.24709474e+01]
 [ 3.11176842e+01]
 [ 3.10130526e+01]
 [ 1.17825737e+01]
 [ 1.23770526e+01]
 [ 1.62517895e+01]
 [ 3.02431579e+01]
 [ 2.55372632e+01]
 [ 3.69335789e+01]
 [ 4.06950526e+01]
 [ 3.40103158e+01]
 [ 1.36309211e+01]
 [ 1.41655526e+01]
 [ 1.41875702e+01]
 [ 1.41255789e+01]
 [ 1.42276842e+01]
 [ 1.38273684e+01]
 [ 2.98757895e+01]
 [ 2.62036842e+01]
 [ 2.56151579e+01]
 [ 1.38206316e+01]
 [ 1.38620263e+01]
 [ 1.42724211e+01]
 [ 1.92309474e+01]
 [ 1.93350526e+01]
 [ 1.93081053e+01]
 [ 1.42935789e+01]
 [ 3.20562105e+01]
 

In [14]:
print ("RF y1", np.mean(result_f_y1))
print ("RF y2", np.mean(result_f_y2))

RF y1 0.22911222161272696
RF y2 2.2057171888322578


In [15]:
from sklearn.neighbors import KNeighborsRegressor
model_k = KNeighborsRegressor()
params = {'n_neighbors': list(range(1,27,2))}
result_k_y1, values_k_y1 = closs_val_test(model_k, params, x_trans, y1_trans, ss_y1)
result_k_y2, values_k_y2 = closs_val_test(model_k, params, x_trans, y2_trans, ss_y2)

best_model: KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=17, p=2,
          weights='uniform')
output_values: [[21.63411765]
 [21.4       ]
 [22.54294118]
 [20.84352941]
 [19.69647059]
 [19.02823529]
 [21.18235294]
 [ 7.56941176]
 [ 7.04529412]
 [ 8.66117647]
 [ 8.74117647]
 [ 8.72294118]
 [28.35058824]
 [28.66      ]
 [28.76823529]
 [28.66      ]
 [29.04294118]
 [28.92882353]
 [30.42823529]
 [32.80235294]
 [32.73588235]
 [12.92058824]
 [13.35470588]
 [13.35117647]
 [13.12176471]
 [13.06352941]
 [13.17764706]
 [28.52823529]
 [28.47823529]
 [28.09058824]
 [28.39      ]
 [31.03705882]
 [33.46941176]
 [12.87823529]
 [12.92588235]
 [13.27647059]
 [29.65529412]
 [29.41764706]
 [30.83411765]
 [11.93588235]
 [11.66588235]
 [11.68176471]
 [12.03529412]
 [13.05117647]
 [13.31058824]
 [13.19058824]
 [27.95764706]
 [28.88294118]
 [30.62588235]
 [29.71294118]
 [31.78882353]
 [12.02705882]
 [11.65823529]
 [12.42117647]
 

best_model: KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=17, p=2,
          weights='uniform')
output_values: [[20.53941176]
 [21.20588235]
 [20.48294118]
 [21.62647059]
 [20.23764706]
 [20.40411765]
 [24.63941176]
 [ 8.11470588]
 [ 8.06823529]
 [ 7.74882353]
 [ 7.75176471]
 [28.08294118]
 [29.41352941]
 [29.84294118]
 [28.53117647]
 [33.08294118]
 [31.78823529]
 [11.70705882]
 [12.53647059]
 [27.77058824]
 [30.40117647]
 [30.62411765]
 [11.89176471]
 [12.24411765]
 [12.80235294]
 [13.20470588]
 [28.21647059]
 [27.60588235]
 [30.04470588]
 [31.80647059]
 [31.31294118]
 [11.66411765]
 [12.86235294]
 [13.52235294]
 [29.03705882]
 [29.36      ]
 [28.81529412]
 [28.38411765]
 [31.82058824]
 [11.97647059]
 [13.18      ]
 [12.96823529]
 [13.44117647]
 [13.10529412]
 [28.38823529]
 [27.29176471]
 [29.06529412]
 [30.70058824]
 [29.06588235]
 [32.64588235]
 [11.71882353]
 [11.38117647]
 [12.01823529]
 [13.07411765]
 

best_model: KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=17, p=2,
          weights='uniform')
output_values: [[26.43764706]
 [25.88058824]
 [27.31529412]
 [25.97470588]
 [24.73      ]
 [24.09294118]
 [26.56588235]
 [12.55      ]
 [11.99588235]
 [13.67705882]
 [13.31411765]
 [13.27941176]
 [31.10529412]
 [29.80764706]
 [32.07705882]
 [29.80764706]
 [32.48117647]
 [31.13117647]
 [32.84235294]
 [33.68764706]
 [35.37647059]
 [15.98058824]
 [16.35470588]
 [16.34058824]
 [16.08176471]
 [15.54882353]
 [15.71764706]
 [31.26176471]
 [31.43647059]
 [30.65588235]
 [29.59058824]
 [32.68588235]
 [35.67705882]
 [15.97588235]
 [16.05235294]
 [15.67647059]
 [31.24823529]
 [31.80058824]
 [33.14647059]
 [14.82117647]
 [14.63882353]
 [14.32058824]
 [14.82705882]
 [15.85      ]
 [16.34882353]
 [16.21882353]
 [30.50764706]
 [31.81176471]
 [33.14470588]
 [32.22764706]
 [34.32      ]
 [15.07647059]
 [14.68764706]
 [15.33352941]
 

best_model: KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=1, p=2,
          weights='uniform')
output_values: [[25.16]
 [29.6 ]
 [23.77]
 [21.16]
 [39.44]
 [21.16]
 [31.27]
 [11.27]
 [11.73]
 [11.73]
 [11.93]
 [27.31]
 [29.87]
 [35.28]
 [27.03]
 [27.03]
 [41.68]
 [13.8 ]
 [19.34]
 [25.89]
 [25.02]
 [28.79]
 [13.8 ]
 [13.8 ]
 [19.23]
 [19.32]
 [24.93]
 [43.12]
 [27.54]
 [43.12]
 [35.1 ]
 [14.14]
 [14.44]
 [14.09]
 [25.91]
 [26.18]
 [25.87]
 [25.16]
 [24.61]
 [14.42]
 [14.29]
 [19.42]
 [14.29]
 [14.29]
 [26.18]
 [27.69]
 [27.33]
 [25.11]
 [27.33]
 [29.52]
 [14.26]
 [14.28]
 [14.39]
 [19.18]
 [19.29]
 [29.79]
 [15.85]
 [15.21]
 [15.85]
 [15.81]
 [15.77]
 [26.96]
 [26.72]
 [29.88]
 [36.77]
 [15.4 ]
 [20.47]
 [15.85]
 [15.64]
 [35.1 ]
 [27.76]
 [43.66]
 [37.76]
 [36.87]
 [15.57]
 [33.16]
 [44.16]
 [15.2 ]
 [19.65]
 [15.38]
 [19.65]
 [26.08]
 [35.62]
 [26.08]
 [30.  ]
 [26.08]
 [28.14]
 [37.54]
 [15.27]
 [14.97]
 [

In [16]:
print ("KNN y1", np.mean(result_k_y1))
print ("KNN y2", np.mean(result_k_y2))

KNN y1 7.295982006632663
KNN y2 11.575257681724148
